# Create individual user matrices (csv files)
Skip these cells

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('dataset_mood_smartphone.csv')

In [2]:
# all users
users = list(df.id.unique())
days_dict = {}
avg_moods_dict = {}

# lack of inspiration...
arousal_dict, valence_dict, activity_dict, screen_dict, call_dict, sms_dict, builtin_dict, \
communication_dict, entertainment_dict, finance_dict, game_dict, office_dict, other_dict, social_dict, \
travel_dict, unknown_dict, utilities_dict, weather_dict = {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, \
                                                            {}, {}, {}, {}, {}, {}

for i in range(len(users)):
    # specify user
    user = df[df.id == users[i]]

    # all days with data for the user
    #days = sorted(set([x.split()[0] for x in list(user['time'])]))

    pd.set_option('display.max_columns', 50)

    # get all days on which the user's mood is known (mood_days)
    mood_days = sorted(set([x.split()[0] for x in list(user[user.variable=="mood"]['time'])]))

    average_moods, average_arousal, average_valence, average_activity, screen_sum, call_sum, sms_sum, \
    builtin_sum, comm_sum, ent_sum, fin_sum, game_sum, off_sum, oth_sum, soc_sum, tra_sum, unk_sum, uti_sum, \
    wea_sum = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [] 
 
    for day in mood_days:
        day_df = user[user['time'].str.contains(day)]
        avg_mood = day_df[day_df.variable=="mood"].value.mean()
        average_moods.append(avg_mood)
        average_arousal.append(day_df[day_df.variable=='circumplex.arousal'].value.mean())
        average_valence.append(day_df[day_df.variable=='circumplex.valence'].value.mean())
        average_activity.append(day_df[day_df.variable=='activity'].value.mean())
        screen_sum.append(day_df[day_df.variable=='screen'].value.sum())
        call_sum.append(day_df[day_df.variable=='call'].value.sum())
        sms_sum.append(day_df[day_df.variable=='sms'].value.sum())
        builtin_sum.append(day_df[day_df.variable=='appCat.builtin'].value.sum())
        comm_sum.append(day_df[day_df.variable=='appCat.communication'].value.sum())
        ent_sum.append(day_df[day_df.variable=='appCat.entertainment'].value.sum())
        fin_sum.append(day_df[day_df.variable=='appCat.finance'].value.sum())
        game_sum.append(day_df[day_df.variable=='appCat.game'].value.sum())
        off_sum.append(day_df[day_df.variable=='appCat.office'].value.sum())
        oth_sum.append(day_df[day_df.variable=='appCat.other'].value.sum())
        soc_sum.append(day_df[day_df.variable=='appCat.social'].value.sum())
        tra_sum.append(day_df[day_df.variable=='appCat.travel'].value.sum())
        unk_sum.append(day_df[day_df.variable=='appCat.unknown'].value.sum())
        uti_sum.append(day_df[day_df.variable=='appCat.utilities'].value.sum())
        wea_sum.append(day_df[day_df.variable=='appCat.weather'].value.sum())
        
    days_dict[users[i]] = mood_days
    avg_moods_dict[users[i]] = average_moods
    arousal_dict[users[i]] = average_arousal
    valence_dict[users[i]] = average_valence
    activity_dict[users[i]] = average_activity
    screen_dict[users[i]] = screen_sum
    call_dict[users[i]] = call_sum
    sms_dict[users[i]] = sms_sum
    builtin_dict[users[i]] = builtin_sum
    communication_dict[users[i]] = comm_sum
    entertainment_dict[users[i]] = ent_sum
    finance_dict[users[i]] = fin_sum
    game_dict[users[i]] = game_sum
    office_dict[users[i]] = off_sum
    other_dict[users[i]] = oth_sum
    social_dict[users[i]] = soc_sum
    travel_dict[users[i]] = tra_sum
    unknown_dict[users[i]] = unk_sum
    utilities_dict[users[i]] = uti_sum
    weather_dict[users[i]] = wea_sum

In [3]:
# @cornelis
del df

In [4]:
import matplotlib.pyplot as plt

users = list(days_dict)
plt.plot(range(len(days_dict[users[0]])), office_dict[users[0]])
plt.show()

In [37]:
# create csv files for each user (average mood, arousal, valence, activity and sum of other variables)
# located under '/data_mounir'

users = list(days_dict)
dicts = [avg_moods_dict, arousal_dict, valence_dict, activity_dict, screen_dict, \
         call_dict, sms_dict, builtin_dict, communication_dict, entertainment_dict, finance_dict, \
         game_dict, office_dict, other_dict, social_dict, travel_dict, unknown_dict, utilities_dict, weather_dict] 

for user in users:
    userframe = pd.DataFrame(index=days_dict[user] , columns=['mood', 'arousal', 'valence', 'activity', \
                                                                  'screen', 'calls', 'sms', 'builtin', 
                                                                  'communication', 'entertainment', 'finance', \
                                                                  'game', 'office', 'other', 'social', \
                                                                  'travel', 'unknown', 'utilities', 'weather'])
        
    for i in range(len(userframe.columns)):
        col = userframe.columns[i]
        userframe[col] = dicts[i][user] 
    
    userframe.to_csv('data_mounir/' + user + '.csv')


# Create instance based dataset
Generate time window based dataset (for now only mood, arousal, valence and activity are used)

In [86]:
import os
import pandas as pd
import datetime
import numpy as np

user_matrices = os.listdir('data_mounir')

# define a time window
window = 5

# initialize final dataframe
dataset_df = pd.DataFrame(columns=['mood', 'arousal', 'valence', 'activity', 'predicted_mood'])

# use a single user for now
usermatrix = user_matrices[3]
userdf = pd.read_csv(os.path.join('data_mounir', usermatrix), index_col=0)
days = list(userdf.index)

# convert days to datetime objects
dates = []
for day in days:
    day_ints = [int(x) for x in day.split('-')]
    dates.append(datetime.date(day_ints[0], day_ints[1], day_ints[2]))

# check that the days in a time window are consecutive    
length = len(dates) - 1
for i in range(len(dates)):
    # make sure to not exceed the size of the days list when indexing
    if i + window <= length: 
        if (dates[i+window] - dates[i]).days == window:
            # reset average values
            avg_activity, avg_mood, avg_arousal, avg_valence = [np.NaN] * 4
            
            # check if there's missing values in window and if complete average
            if not any(list(pd.isnull(userdf[i:i+window]['mood']))):
                avg_mood = userdf[i:i+window]['mood'].mean()    

            if not any(list(pd.isnull(userdf[i:i+window]['arousal']))):
                avg_arousal = userdf[i:i+window]['arousal'].mean()    
            
            if not any(list(pd.isnull(userdf[i:i+window]['valence']))):
                avg_valence = userdf[i:i+window]['valence'].mean()    

            if not any(list(pd.isnull(userdf[i:i+window]['activity']))):
                avg_activity = userdf[i:i+window]['activity'].mean()    
                            
            row = pd.Series([avg_mood, avg_arousal, avg_valence, avg_activity])

            if not any(list(pd.isnull(row))):
                dataset_df = dataset_df.append({'mood': avg_mood, 'arousal': avg_arousal, \
                                'valence': avg_valence, 'activity': avg_activity, \
                                'predicted_mood': userdf.loc[days[i+window]]['mood']}, ignore_index=True)

        else:
            #print "bad: {} {}\n".format(dates[i], dates[i+window])
            pass
    else:
        pass    

In [87]:
dataset_df

,mood,arousal,valence,activity,predicted_mood
0,6.940000,0.370000,0.790000,0.099782,3.750000
1,6.490000,0.050000,0.510000,0.086821,4.400000
2,5.870000,-0.280000,0.110000,0.081230,3.750000
3,5.270000,-0.880000,-0.240000,0.055799,5.333333
4,4.896667,-1.280000,-0.333333,0.040301,5.500000
5,4.546667,-1.380000,-0.483333,0.038109,5.800000
6,4.956667,-1.100000,-0.243333,0.049832,6.200000
7,5.316667,-0.820000,0.076667,0.041848,6.400000
8,5.846667,-0.500000,0.476667,0.061425,6.200000
9,6.020000,-0.460000,0.570000,0.068216,6.800000
